In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
SCOPETYPE = 'OPENvADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
def load_bitstream(bitstream_file):
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.setFPGAMode("debug")
    scope.scopetype.cwFirmwareConfig[0xACE2].loader._bsLoc = bitstream_file
    scope.scopetype.cwFirmwareConfig[0xACE2].loader.save_bsLoc()
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader._release_mode))
    print("Mode: " + str(scope.scopetype.cwFirmwareConfig[0xACE2].loader.fpga_bitstream()))
    input("powercycle done?")
    %run "../Setup_Scripts/Setup_Generic.ipynb"
    
load_bitstream("../../hardware/capture/chipwhisperer-lite/cwlite_interface_ec_256.bit")

In [ ]:
import holoviews as hv
import pandas as pd
import numpy as np

from holoviews.streams import Pipe, Buffer
from scipy.ndimage.filters import uniform_filter1d
from tqdm.notebook import trange

import chipwhisperer.analyzer as cwa
import chipwhisperer as cw
from chipwhisperer.common.traces import Trace
import edge_counter

class real_time_plot:
    def __init__(self, plot_len, title="Plot"):
        hv.extension('bokeh')
        st = pd.DataFrame({'y':[]}, columns=['y'])
        self.dfstream = Buffer(st, length=plot_len, index=False)
        curve_dmap = hv.DynamicMap(hv.Curve, streams=[self.dfstream])
        curve_dmap.opts(tools=["hover"], width=800, height=600, title=title)
        display(curve_dmap)
        self.curve = curve_dmap
    def send(self, data):
        if hasattr(data, 'wave'):
            d = data.wave
        else:
            d = data
        self.dfstream.send(pd.DataFrame({'y':d}))

def powercycle_target(scope):
    scope.io.tio1 = "gpio_low"
    time.sleep(0.0001)
    scope.io.tio1 = "gpio_high"
    
def print_ec_conf():
    print("==== Trigger ====")
    print(scope.EC)
    
def setup_cw():    
    scope.adc.offset = offset
    scope.gain.gain = 50
    scope.adc.samples = nr_samples
    scope.adc.presamples = 2000
    
    # per docs has to be like that for DecodeIO and SAD
    scope.adc.basic_mode = "rising_edge"
    
def setup_glitcher():
    scope.glitch.width = 0.2
    scope.glitch.width_fine = 200
    scope.glitch.trigger_src = "ext_single"
    scope.glitch.repeat = 1
    scope.glitch.output = "glitch_only"

def setup_ec():
    setup_cw()
    scope.EC.reset()
    scope.EC.window_size = 50
    scope.EC.threshold = 0.135
    scope.EC.hold_cycles = 1
    scope.EC.absolute_values = True
    scope.EC.edge_num = 1
    scope.EC.edge_type = "rising_edge"
    scope.EC.start()
    scope.trigger.module = "EC"

def setup_ec_offline():
    settling_time = 100
    trig_threshold = 0.16
    edge_num = 1
    edge_type="rising_edge"
    pretrigger_ctr=1
    
    return edge_counter.edge_count(settling_time, trig_threshold, edge_num, edge_type, pretrigger_ctr)
    
def collect_trace():
    powercycle_target(scope)
    scope.arm()
    scope.capture()
    return scope.get_last_trace()
    
# init plotting        
hv.extension('bokeh')

nr_samples = 24400  
offset = 144000
traces = 5
ec_offline = setup_ec_offline()

setup_ec()
print_ec_conf()

# Run EC trigger

trace = collect_trace()
orig_curve = hv.Curve(trace).opts(tools=["hover"], width=800, height=600, title="orig", axiswise=True)
trace_sum = ec_offline._movsum(trace)
movsum_curve = hv.Curve(trace_sum).opts(tools=["hover"], width=800, height=600, title="sum", axiswise=True)
trace_avg = ec_offline._movavg(trace)
movavg_curve = hv.Curve(trace_avg).opts(tools=["hover"], width=800, height=600, title="avg", axiswise=True)
(orig_curve)+(movsum_curve)+(movavg_curve)

In [ ]:
def setup_basic():
    setup_cw()

    scope.adc.basic_mode = "low"
    scope.trigger.module = "basic"
    scope.trigger.triggers = "tio4"

# Show a longer trace to get a sense of what's going on in the target
def collect_reference_trace(traces, nr_samples, offset):
    total_len = traces*nr_samples

    new_trace = np.zeros(0)
    for i in trange(traces, desc='Capturing traces'):
        cur_offset = int(offset + nr_samples*i)
        scope.adc.offset = cur_offset

        scope.arm()
        powercycle_target(scope)
        scope.capture()

        trace = scope.get_last_trace()
        if trace is None:
            continue
        new_trace = np.append(new_trace, trace)
    
    return new_trace

def display_current_capture():
    long_trace = collect_reference_trace(traces, nr_samples, offset)
    
    trace_sum = ec_offline._movsum(long_trace, True)
    trace_avg = ec_offline._movavg(long_trace, True)
    
    orig_curve = hv.Curve(long_trace).opts(tools=["hover"], width=800, height=600, title="orig", axiswise=True)
    movsum_curve = hv.Curve(trace_sum).opts(tools=["hover"], width=800, height=600, title="sum", axiswise=True)
    movavg_curve = hv.Curve(trace_avg).opts(tools=["hover"], width=800, height=600, title="avg", axiswise=True)
    
    return (orig_curve, movsum_curve, movavg_curve)

def build_project(name):
    project = cw.create_project(name)

    for i in range(0, 9):
        long_trace = collect_reference_trace(traces, nr_samples, offset)
        project.traces.append(Trace(long_trace, None, None, None))

    project.save()

def display_project(name):
    project = cw.open_project(name)
    
    for trace in project.traces:
        pass

nr_samples = 24400  
offset = 0
traces = 20

setup_basic()

orig_curve, movsum_curve, movavg_curve = display_current_capture()
(orig_curve)+(movsum_curve)+(movavg_curve)
# mult_traces = np.ndarray((2, nr_samples))
# for i in range (0, 2):
# long_trace = collect_reference_trace(traces, nr_samples, offset)
#     np.append(mult_traces, long_trace)

# for trace in mult_traces:

# build_project("projects/working_mb2")

# display_project("projects/working_mb2")